# HACHKATON AirBus
Ce notebook est le fruit du travail de:

|Nom | Prénom|
|---|---|
| AHOUNOU | Méryl |
| DJIMA | Saleem |
| ILBOUDO| BEN |
| SORHO | Michaela |


Les missions de cet hackathon sont les suivantes:

    La première mission est de détecter le numéro d'immatriculation à partir de photos d'hélicoptères. Vous allez entraîner votre modèle avec les dossiers train et validation. Ensuite, vous le testerez avec le dossier test. Tous les labels correspondant aux images des 3 dossiers sont dans le dossier label. 
    Pour mesurer la performance de votre modèle, deux métriques seront utilisées : la précision moyenne (MAP) et la précision moyenne (AP).
    Pour cette mission, vous livrerez :
    -un fichier csv contenant les coordonnées du numéro d'immatriculation prédit sur l'image (ID, et coordonnées) pour les images de test.
    -un fichier csv contenant l'ID de l'image et la précision moyenne (MAP) et la précision moyenne (AP) pour chaque photo de test
    -un notebook ou un code python permettant d'exécuter votre modèle.

    Le deuxième objectif de la mission est de prendre les images de la source de données Jetson (les ID de ces images sont détaillées dans le fichier de données d'enregistrement). Pour toutes ces données, vous allez extraire le numéro d'enregistrement grâce à votre modèle de la première mission. Ensuite, vous allez essayer de lire le numéro d'enregistrement !
    Pour mesurer la performance de votre méthode, la métrique sera la suivante : Nombre de numéros d'immatriculation exacts lus / Nombre de numéros d'immatriculation à lire.
    Pour cette mission, vous livrerez :
    -un fichier csv contenant l'identifiant de l'image des données jetson et le numéro d'enregistrement prédit / lu.
    -un notebook ou un fichier python permettant d'exécuter votre modèle.

Installation des packages

In [1]:
!pip install imutils

In [2]:
!pip install map-boxes

In [3]:
!pip install easyocr

In [4]:
!pip install mean_average_precision

In [5]:
!pip install --upgrade git+https://github.com/bes-dev/mean_average_precision.git

In [6]:
!pip install opencv-contrib-python==3.4.13.47 --force-reinstall

In [ ]:
from google.colab import drive
import cv2
import matplotlib.pyplot as plt
import os
import xml.etree.ElementTree as et
import random
import argparse
import numpy as np
import pandas as pd
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
import imutils
import easyocr
from skimage.io import imread, imshow
from mean_average_precision import MetricBuilder
from map_boxes import mean_average_precision_for_boxes

In [ ]:
# On a connecté Google Colab au drive pour qu'il puisse accéder aux images et sauvegarder les résultats
drive.mount('/content/drive')

# Mission 1: Détection des numéros d'immatriculation

On a récupérer les coordonnées des images pour le train et la validation du fichier label sous format qu'on a utilisé pour générer un fichier txt contenant les coordonnées des détections sous format YOLO

In [ ]:
# ##### A N'EXECUTER QU'UNE SEULE FOIS
# # génération des fichiers txt pour les coordonnées (train)
# path_img = '/content/drive/MyDrive/hackathon/train'  # changer le chemin en fonction du répertoire ou se trouvent les images train
# path_lab = '/content/drive/MyDrive/hackathon/labels' # changer le chemin en fonction du répertoire ou se trouvent les labels
# input_dir = os.listdir(path_img)
# for input in input_dir:
#   txt_name = input.replace('.jpg', '.txt')
#   txt_path = path_img + '/' + txt_name
#   with open(txt_path, 'w') as f:
#     if input[len(input)-4:len(input)] != '.txt':
#       file = input
#       file_lab = input.replace('.jpg', '.xml')
#       img = cv2.imread(os.path.join(path_img,input))
#       lab_tree = et.parse(os.path.join(path_lab,file_lab))
#       root = lab_tree.getroot()
#       for boxs in root.findall('object'):
#         for box in boxs.findall('bndbox'):
#           x1 = int(box.find('xmin').text)
#           y1 = int(box.find('ymin').text)
#           x2 = int(box.find('xmax').text)
#           y2 = int(box.find('ymax').text)
#           width = int(root[4][0].text)
#           height = int(root[4][1].text)
#           coordinates = str(0) + ' '  + str((x2+x1)/(2*width)) + ' ' + str((y2+y1)/(2*height)) + ' ' + str((x2-x1)/width) + ' ' + str((y2-y1)/height)+ '\n'
#           f.write(coordinates)

In [ ]:
# ##### A N'EXECUTER QU'UNE SEULE FOIS
# # génération des fichiers txt pour les coordonnées (valid)
# path_img = '/content/drive/MyDrive/hackathon/valid' # changer le chemin en fonction du répertoire ou se trouvent les images de validation
# path_lab = '/content/drive/MyDrive/hackathon/labels' # changer le chemin en fonction du répertoire ou se trouvent les labels
# input_dir = os.listdir(path_img)
# for input in input_dir:
#   txt_name = input.replace('.jpg', '.txt')
#   txt_path = path_img + '/' + txt_name
#   with open(txt_path, 'w') as f:
#     if input[len(input)-4:len(input)] != '.txt':
#       file = input
#       file_lab = input.replace('.jpg', '.xml')
#       img = cv2.imread(os.path.join(path_img,input))
#       lab_tree = et.parse(os.path.join(path_lab,file_lab))
#       root = lab_tree.getroot()
#       for boxs in root.findall('object'):
#         for box in boxs.findall('bndbox'):
#           x1 = int(box.find('xmin').text)
#           y1 = int(box.find('ymin').text)
#           x2 = int(box.find('xmax').text)
#           y2 = int(box.find('ymax').text)
#           width = int(root[4][0].text)
#           height = int(root[4][1].text)
#           coordinates = str(0) + ' '  + str((x2+x1)/(2*width)) + ' ' + str((y2+y1)/(2*height)) + ' ' + str((x2-x1)/width) + ' ' + str((y2-y1)/height)+ '\n'
#           f.write(coordinates)

In [ ]:
# On affiche aléatoirement des images pour visualiser
path_img = '/content/drive/MyDrive/hackathon/train'
path_lab = '/content/drive/MyDrive/hackathon/labels'
input_dir = os.listdir(path_img)
for i in range(20):
  file = random.choice(input_dir)
  if file[len(file)-4:len(file)] != '.txt':
    file_lab = file.replace('.jpg', '.xml')
    img = cv2.imread(os.path.join(path_img,file))
    lab_tree = et.parse(os.path.join(path_lab,file_lab))
    root = lab_tree.getroot()
    print(file)
    print(img.shape)
    plt.figure(figsize=(15,40))
    for boxs in root.findall('object'):
      for box in boxs.findall('bndbox'):
        x1 = int(box.find('xmin').text)
        y1 = int(box.find('ymin').text)
        x2 = int(box.find('xmax').text)
        y2 = int(box.find('ymax').text)
        print(x1,y1,x2,y2)
        cv2.rectangle(img, (x1,y1), (x2,y2), (10,35,246), 4)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()
    print('\n')

In [ ]:
# Nous permet de visualiser une image précisément
path_img = '/content/drive/MyDrive/hackathon/train'
path_lab = '/content/drive/MyDrive/hackathon/labels'
file = 'ID_470484_58756_1115745411.jpg'
file_lab = file.replace('.jpg', '.xml')
img = cv2.imread(os.path.join(path_img,file))
lab_tree = et.parse(os.path.join(path_lab,file_lab))
root = lab_tree.getroot()
print(file)
print(img.shape)
plt.figure(figsize=(10,20))
for boxs in root.findall('object'):
  for box in boxs.findall('bndbox'):
    x1 = int(box.find('xmin').text)
    y1 = int(box.find('ymin').text)
    x2 = int(box.find('xmax').text)
    y2 = int(box.find('ymax').text)
    print(x1,y1,x2,y2)
    cv2.rectangle(img, (x1,y1), (x2,y2), (10,35,246), 4)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

# YOLO

Pour la détection des immatriculations, nous avons décider d'utiliser le modèle YOLO 4 qui donne de bons résulats et est beaucoup plus rapides que les modèles par région (R-CNN, Fast R-CNN, Faster R-CNN...). Notre modèle a été réalisé par transfert learning.

In [ ]:
# création du répertoire YOLO
path_yolo = '/content/drive/MyDrive/hackathon/yolo'
if not os.path.exists(path_yolo):
       os.mkdir(path_yolo)

In [ ]:
# # On récupère la configuration du réseau de neurone
# !wget https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4.cfg

In [ ]:
# # on récupère le poids du réseau de neurone
# !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
# # on récupère le fichier qui nous permet de calculer le poids pour notre réseau
# !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
# # on copie les fichiers dans le répertoire YOLO
# !cp '/content/yolov4.cfg'  /content/drive/MyDrive/hackathon/yolo
# !cp '/content/yolov4.weights'  /content/drive/MyDrive/hackathon/yolo
# !cp '/content/yolov4.conv.137'  /content/drive/MyDrive/hackathon/yolo

In [ ]:
%cd '/content/drive/MyDrive/hackathon/yolo'

In [ ]:
# !git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
!chmod +rwx ./*
!make

In [ ]:
def display(_image):
  img1 = imread(_image)
  img2 = imread('predictions.jpg')
  path_lab = '/content/drive/MyDrive/hackathon/labels'
  file_name = _image.split('/')[-1]
  file_lab = os.path.join(path_lab,file_name)
  lab_path = file_lab.replace('.jpg', '.xml')
  lab_tree = et.parse(lab_path)
  root = lab_tree.getroot()
  for boxs in root.findall('object'):
    for box in boxs.findall('bndbox'):
      x1 = int(box.find('xmin').text)
      y1 = int(box.find('ymin').text)
      x2 = int(box.find('xmax').text)
      y2 = int(box.find('ymax').text)
      print(x1,y1,x2,y2)
      cv2.rectangle(img1, (x1,y1), (x2,y2), (10,35,246), 4)
  fig, axes = plt.subplots(ncols=2)
  fig.set_size_inches(18.5, 10.5)
  axes[0].set_axis_off()
  axes[0].imshow(img1)
  axes[1].set_axis_off()
  axes[1].imshow(img2)
  plt.tight_layout()

Génération de fichier txt répertoriant tous les chemins des images train, test et validation

In [ ]:
# from os import listdir
# from os.path import isfile, join
# train_directory = '/content/drive/MyDrive/hackathon/train/'
# valid_directory = '/content/drive/MyDrive/hackathon/valid/'
# test_directory = '/content/drive/MyDrive/hackathon/test/'
# allfiles_train = [f for f in listdir(train_directory) if isfile(join(train_directory, f))]
# allfiles_valid = [f for f in listdir(valid_directory) if isfile(join(valid_directory, f))]
# allfiles_test = [f for f in listdir(test_directory) if isfile(join(test_directory, f))]

# images_files_train = []
# images_files_valid = []
# images_files_test = []
 
# # put all the images files into a list
# for file in allfiles_train:
#   if (file[len(file)-4:len(file)] != ".txt"):
#     images_files_train.append(train_directory +  file)

# for file in allfiles_valid:
#   if (file[len(file)-4:len(file)] != ".txt"):
#     images_files_valid.append(valid_directory +  file)

# for file in allfiles_test:
#   if (file[len(file)-4:len(file)] != ".txt"):
#     images_files_test.append(test_directory +  file)
 
# path_doc = '/content/drive/MyDrive/hackathon/yolo/darknet/doc_data'
# if not os.path.exists(path_doc):
#        os.mkdir(path_doc)

# with open('/content/drive/MyDrive/hackathon/yolo/darknet/doc_data/document_valid.txt', 'w') as f:
#     for item in images_files_valid:
#         f.write("%s\n" % item)
 
# with open('/content/drive/MyDrive/hackathon/yolo/darknet/doc_data/document_training.txt', 'w') as f:
#     for item in images_files_train:
#         f.write("%s\n" % item)

# with open('/content/drive/MyDrive/hackathon/yolo/darknet/doc_data/document_test.txt', 'w') as f:
#     for item in images_files_test:
#         f.write("%s\n" % item)

Entrainement du modèle sur notre jeu de données

In [ ]:
# !./darknet detector train doc_data/document.data '/content/drive/MyDrive/hackathon/yolo/airbus_yolov4.cfg' -map '/content/drive/MyDrive/hackathon/yolo/yolov4.conv.137' -dont_show

In [ ]:
# # cas d'erreur / arret
# !./darknet detector train doc_data/document.data '/content/drive/MyDrive/hackathon/yolo/airbus_yolov4.cfg' backup/airbus_yolov4_last.weights -dont_show -map

In [ ]:
# Prédiction sur une image précise
!./darknet detector test doc_data/document.data '/content/drive/MyDrive/hackathon/yolo/airbus_yolov4.cfg'  '/content/drive/MyDrive/hackathon/yolo/darknet/backup/airbus_yolov4_final.weights'  '/content/drive/MyDrive/hackathon/test/ID_6448067_44579_1230347162.jpg'

In [ ]:
# visualisation de la prédiction
image = '/content/drive/MyDrive/hackathon/test/ID_6448067_44579_1230347162.jpg'
display(image)

In [ ]:
# Prédiction sur une image précise
!./darknet detector test doc_data/document.data '/content/drive/MyDrive/hackathon/yolo/airbus_yolov4.cfg'  '/content/drive/MyDrive/hackathon/yolo/darknet/backup/airbus_yolov4_final.weights'  '/content/drive/MyDrive/hackathon/test/ID10893_Mi17_RA-25748_4.jpg'

In [ ]:
# visualisation de la prédiction
image = '/content/drive/MyDrive/hackathon/test/ID10893_Mi17_RA-25748_4.jpg'
display(image)

In [ ]:
# Prédiction sur une image précise
!./darknet detector test doc_data/document.data '/content/drive/MyDrive/hackathon/yolo/airbus_yolov4.cfg'  '/content/drive/MyDrive/hackathon/yolo/darknet/backup/airbus_yolov4_final.weights'  '/content/drive/MyDrive/hackathon/test/ID6721_Mi17_RA-22429_1.jpg'

In [ ]:
# visualisation de la prédiction
image = '/content/drive/MyDrive/hackathon/test/ID6721_Mi17_RA-22429_1.jpg'
display(image)

In [ ]:
path_test_prediction = '/content/drive/MyDrive/hackathon/test_prediction'
if not os.path.exists(path_test_prediction):
       os.mkdir(path_test_prediction)

In [ ]:
# visualisation aléatoirement des images test
path_test = '/content/drive/MyDrive/hackathon/test/'
input_dir_test = os.listdir(path_test)
for i in range(20):
  file = random.choice(input_dir_test)
  if file[len(file)-4:len(file)] != '.txt':
    print(file)
    img = os.path.join(path_test,file)
    print(img)
    with open('/content/drive/MyDrive/hackathon/yolo/test_file.txt', 'w') as f:
      f.write(img)   
    # plus de 30% 
    !./darknet detector test doc_data/document.data '/content/drive/MyDrive/hackathon/yolo/airbus_yolov4.cfg'  'backup/airbus_yolov4_final.weights' <'/content/drive/MyDrive/hackathon/yolo/test_file.txt'  #-thresh 0.3
    display(img)
    plt.show()
    print('\n')

In [ ]:
# # Visualisation et génération des prédictions pour tous les images test
# path_test = '/content/drive/MyDrive/hackathon/test/'
# input_dir_test = os.listdir(path_test)
# for file in input_dir_test:
#   if file[len(file)-4:len(file)] != '.txt':
#     # print(file)
#     img = os.path.join(path_test,file)
#     with open('/content/drive/MyDrive/hackathon/yolo/test_file.txt', 'w') as f:
#       f.write(img)   
#     # plus de 30% 
#     !./darknet detector test doc_data/document.data '/content/drive/MyDrive/hackathon/yolo/airbus_yolov4.cfg'  'backup/airbus_yolov4_final.weights' <'/content/drive/MyDrive/hackathon/yolo/test_file.txt' #-thresh 0.3
#     img2 = imread('predictions.jpg')
#     plt.imshow(img2)
#     path_save = os.path.join(path_test_prediction, file)
#     plt.savefig(path_save)

In [ ]:
# # Génère un fichier txt contenant les résulats des prédictions des images test
# !./darknet detector test doc_data/document.data '/content/drive/MyDrive/hackathon/yolo/airbus_yolov4.cfg'  'backup/airbus_yolov4_final.weights'  -dont_show -ext_output < doc_data/document_test.txt >  '/content/drive/MyDrive/hackathon/yolo/result.txt'

# Extraction des coordonnées des prédictions sur les images

On cherche à créer des listes contenant les coordonnées des prédictions sous format YOLO et sous le format xml de base. Avec ces listes , nous générerons des dataframes. 
On vérifie aussi, si le nomre de prédictions dans les labels est le meme que le nombre de prédiction du modèle afin de pouvoir calculer les métriques(AP, mAP).

In [ ]:
# création d'une fonction qui permet d'accéder aux sorties d'une prédiction
def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

scale = 0.00392
classes = None
with open('/content/drive/MyDrive/hackathon/yolo/classes.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

In [ ]:
list_coord = []
list_coord2 = []
label = 'Registration'
paths = '/content/drive/MyDrive/hackathon/test'
path_img = '/content/drive/MyDrive/hackathon/test'
path_lab = '/content/drive/MyDrive/hackathon/labels'
for file in os.listdir(paths):
  if file[len(file)-4:len(file)] != '.txt':
    file_lab = file.replace('.jpg', '.xml')
    lab_tree = et.parse(os.path.join(path_lab,file_lab))
    root = lab_tree.getroot()
  image = cv2.imread(os.path.join(paths,file))
  Width = image.shape[1]
  Height = image.shape[0]
  net = cv2.dnn.readNet('/content/drive/MyDrive/hackathon/yolo/darknet/backup/airbus_yolov4_final.weights', '/content/drive/MyDrive/hackathon/yolo/airbus_yolov4.cfg')
  blob = cv2.dnn.blobFromImage(image, scale, (640,640), (0,0,0), True, crop=False)
  net.setInput(blob)
  outs = net.forward(get_output_layers(net))
  class_ids = []
  confidences = []
  boxes = []
  conf_threshold = 0.3
  nms_threshold = 0.4
  for out in outs:
      for detection in out:
          scores = detection[5:]
          class_id = np.argmax(scores)
          confidence = scores[class_id]
          if confidence > 0.2:
              center_x = int(detection[0] * Width)
              center_y = int(detection[1] * Height)
              w = int(detection[2] * Width)
              h = int(detection[3] * Height)
              x = center_x - w / 2
              y = center_y - h / 2
              class_ids.append(class_id)
              confidences.append(float(confidence))
              boxes.append([round(x), round(y), round(w), round(h)])
  indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
  for i in indices:
    i = i[0]
    confidence = confidences[i]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    coord = file + ' ' + str(confidence) + ' ' + str(x/Width) + ' ' + str(y/Height) + ' ' +str((x+w)/Width) + ' ' + str((y+h)/Height)
    coord2 = file + ' ' + label +  ' ' + str(confidence) + ' ' + str(x/Width) + ' ' + str(y/Height) + ' ' +str((x+w)/Width) + ' ' + str((y+h)/Height)
    list_coord.append(coord)
    list_coord2.append(coord2)

In [ ]:
dict = {'ID': [x.split()[0] for x in list_coord],
        'Confidence': [x.split()[1] for x in list_coord],
        'xmin': [x.split()[2] for x in list_coord],
        'ymin': [x.split()[3] for x in list_coord],
        'xmax': [x.split()[4] for x in list_coord],
        'ymax': [x.split()[5] for x in list_coord]}
df = pd.DataFrame(dict)
df

In [ ]:
df.to_csv('/content/drive/MyDrive/hackathon/yolo/registration.csv')

# Calcul des métriques

In [ ]:
dict_det = {'ImageID': [x.split()[0] for x in list_coord2],
            'LabelName': [x.split()[1] for x in list_coord2],
            'Conf': [x.split()[2] for x in list_coord2],
            'XMin': [x.split()[3] for x in list_coord2],
            'YMin': [x.split()[4] for x in list_coord2],
            'XMax': [x.split()[5] for x in list_coord2],
            'YMax': [x.split()[6] for x in list_coord2]}
det = pd.DataFrame(dict_det)
det

In [ ]:
path_img = '/content/drive/MyDrive/hackathon/test'
path_lab = '/content/drive/MyDrive/hackathon/labels'
input_dir = os.listdir(path_img)
list_coor = []
label = 'Registration'
for file in input_dir:
  if file[len(file)-4:len(file)] != '.txt':
    file_lab = file.replace('.jpg', '.xml')
    lab_tree = et.parse(os.path.join(path_lab,file_lab))
    root = lab_tree.getroot()
    for boxs in root.findall('object'):
      for box in boxs.findall('bndbox'):
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        w = int(root[4][0].text)
        h = int(root[4][1].text)
        coord = file + ' ' + label +  ' ' + str(xmin/w) + ' ' + str(ymin/h) + ' ' +str(xmax/w) + ' ' + str(ymax/h)
        list_coor.append(coord)

In [ ]:
dict_ann = {'ImageID': [x.split()[0] for x in list_coor],
            'LabelName': [x.split()[1] for x in list_coor],
            'XMin': [x.split()[2] for x in list_coor],
            'YMin': [x.split()[3] for x in list_coor],
            'XMax': [x.split()[4] for x in list_coor],
            'YMax': [x.split()[5] for x in list_coor]}
ann = pd.DataFrame(dict_ann)
ann

In [ ]:
ann1 = ann[['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax']].values
det1 = det[['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax']].values
mean_ap, average_precisions = mean_average_precision_for_boxes(ann1, det1)

In [ ]:
ann['mAP'] = 0
ann['AP'] = 0
for i in range(len(ann)):
  x = ann.loc[i, 'ImageID']
  ann1 = ann[ann['ImageID'] == x ]
  ann1 = ann1[['ImageID', 'LabelName', 'XMin', 'XMax', 'YMin', 'YMax']].values
  det1 = det[det['ImageID'] == x ]
  det1 = det1[['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax']].values
  mean_ap, average_precisions = mean_average_precision_for_boxes(ann1, det1)
  ann.loc[i, 'mAP'] = mean_ap
  ann.loc[i, 'AP'] =  average_precisions['Registration'][0]

In [ ]:
ann

In [ ]:
df_final = ann[['ImageID', 'AP', 'mAP']]
df_final

In [ ]:
df_final.to_csv('/content/drive/MyDrive/hackathon/yolo/registration_metrics.csv')

# OCR

In [ ]:
# path_ocr = '/content/drive/MyDrive/hackathon/ocr'
# if not os.path.exists(path_ocr):
#        os.mkdir(path_ocr)

In [ ]:
df_jetson = pd.read_csv('/content/drive/MyDrive/hackathon/yolo/registration_data.csv')
df_jetson

In [ ]:
df_jetson = df_jetson[df_jetson['Source'] == 'Jetson']
df_jetson

In [ ]:
list_img_jetson = []
repertorys = ['train', 'test', 'valid']
for repertory in repertorys:
  path_repertory = '/content/drive/MyDrive/hackathon' + '/' + repertory
  for file in os.listdir(path_repertory):
    if (file[len(file)-4:len(file)] != ".txt"):
      list_img_jetson.append((path_repertory.split('/content/drive/MyDrive/hackathon')[-1]) + '/' + file)
list_img_jetson

In [ ]:
df_jetson = df_jetson.reset_index(drop=True)
df_jetson

In [ ]:
df_jetson['Path'] = None
for i in df_jetson.index:
  x = df_jetson.loc[i, 'Photo_ID']
  for y in list_img_jetson:
    if x in y:
      df_jetson.loc[i, 'Path'] = y 

In [ ]:
df_jetson

In [ ]:
df_jetson[df_jetson['Path'].isna()]

In [ ]:
df_jetson.dropna(subset=['Path'], inplace=True)

In [ ]:
df_jetson = df_jetson.reset_index(drop=True)
df_jetson

In [ ]:
image = cv2.imread('/content/drive/MyDrive/hackathon/train/ID_10236506_17076_1626433515.jpg')

Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392

classes = None

with open('/content/drive/MyDrive/hackathon/yolo/classes.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

net = cv2.dnn.readNet('/content/drive/MyDrive/hackathon/yolo/darknet/backup/airbus_yolov4_final.weights', '/content/drive/MyDrive/hackathon/yolo/airbus_yolov4.cfg')

blob = cv2.dnn.blobFromImage(image, scale, (640,640), (0,0,0), True, crop=False)

net.setInput(blob)

outs = net.forward(get_output_layers(net))
class_ids = []
confidences = []
boxes = []
conf_threshold = 0.3
nms_threshold = 0.4

c = 0
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.2:
              center_x = int(detection[0] * Width)
              center_y = int(detection[1] * Height)
              w = int(detection[2] * Width)
              h = int(detection[3] * Height)
              x = center_x - w / 2
              y = center_y - h / 2
              class_ids.append(class_id)
              confidences.append(float(confidence))
              boxes.append([round(x), round(y), round(w), round(h)])
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
for i in indices:
    i = i[0]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    crop_img = image[y:y+h, x:x+w]
    print(crop_img.shape, image.shape)
    cv2_imshow(image)
    cv2_imshow(crop_img)

In [ ]:
reader = easyocr.Reader(['en'])
result = reader.readtext(crop_img, paragraph="False")
result

In [ ]:
df_jetson['Registration_OCR'] = None
reader = easyocr.Reader(['en'])
for ind in range(len(df_jetson)):
# for ind in range(15):
  x = df_jetson.loc[ind, 'Path']
  x = '/content/drive/MyDrive/hackathon' + x
  image = cv2.imread(x)
  Width = image.shape[1]
  Height = image.shape[0]
  scale = 0.00392
  classes = None
  with open('/content/drive/MyDrive/hackathon/yolo/classes.txt', 'r') as f:
      classes = [line.strip() for line in f.readlines()]
  COLORS = np.random.uniform(0, 255, size=(len(classes), 3))
  net = cv2.dnn.readNet('/content/drive/MyDrive/hackathon/yolo/darknet/backup/airbus_yolov4_final.weights', '/content/drive/MyDrive/hackathon/yolo/airbus_yolov4.cfg')
  blob = cv2.dnn.blobFromImage(image, scale, (640,640), (0,0,0), True, crop=False)
  net.setInput(blob)
  outs = net.forward(get_output_layers(net))
  class_ids = []
  confidences = []
  boxes = []
  conf_threshold = 0.3
  nms_threshold = 0.4
  for out in outs:
      for detection in out:
          scores = detection[5:]
          class_id = np.argmax(scores)
          confidence = scores[class_id]
          if confidence > 0.2:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([round(x), round(y), round(w), round(h)])
  indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
  result_list = []
  if len(indices) == 0:
    continue
  for i in indices:
      i = i[0]
      box = boxes[i]
      x = box[0]
      y = box[1]
      w = box[2]
      h = box[3]
      for x_crop in range(-18, 48, 3):
        for y_crop in range(-18, 48, 3): 
          crop_img = image[y-y_crop:y+h+y_crop, x-x_crop:x+w+x_crop]
          try:
            result = reader.readtext(crop_img, paragraph="False")
            if len(result) == 1:
              result = result[0][1]
              # result = result[0][1][0:6]
            elif len(result) == 2:
              result = result[0][1][0:6] + result[1][1]
              # result = result[0][1] + result[1][1][0:6]
          except:
            result = None
          result_list.append(result)
          # print(result)
          if df_jetson.loc[ind, 'Registration'] in result_list:
            break
        if df_jetson.loc[ind, 'Registration'] in result_list:
          break
      if df_jetson.loc[ind, 'Registration'] in result_list:
        break
  df_jetson['Registration_OCR'][ind] = result_list

In [ ]:
df_jetson.head(25)

In [ ]:
df_jetson[df_jetson['Registration_OCR'].isna()]

In [ ]:
df_jetson[df_jetson['Registration_OCR'].isna()].count()

In [ ]:
number_registration = len(df_jetson)
number_registration

In [ ]:
df_jetson['Read'] = None
df_jetson

In [ ]:
for i in df_jetson.index:
  x = df_jetson.loc[i, 'Registration_OCR']
  y = df_jetson.loc[i, 'Registration']
  if x == None:
    continue
  # print(x)
  if y in x:
    df_jetson.loc[i, 'Read'] = 1
  else:
    df_jetson.loc[i, 'Read'] = 0
df_jetson

In [ ]:
number_read = df_jetson.Read.sum()
number_read

In [ ]:
percentage_read = (number_read /number_registration)*100
percentage_read

In [ ]:
df_jetson = df_jetson.drop(['Path', 'Unnamed: 0'] , axis=1)
df_jetson

In [ ]:
df_jetson.to_csv('/content/drive/MyDrive/hackathon/yolo/registration_data_final.csv')